In [8]:
import pandas as pd
from dotenv import load_dotenv

from os import environ

In [9]:
load_dotenv()

ESRI_USERNAME = environ.get("ESRI_USERNAME")
ESRI_PASSWORD = environ.get("ESRI_PASSWORD")
ESRI_REFERER = environ.get("ESRI_REFERER")

In [10]:
from ladb_data.service import LadbDataService

service = LadbDataService(ESRI_USERNAME, ESRI_PASSWORD, ESRI_REFERER)

In [11]:
rows = service.get_road_segments_by_name("Balonne Highway")

In [12]:
rows[0]

{'road_id': 'QLDRBAL1464501528001575800',
 'segment_id': '16015103990000018315',
 'road_name_full': 'Balonne Highway',
 'alias_1_name_full': None,
 'lga_name_left': 'Paroo Shire',
 'lga_name_right': 'Paroo Shire',
 'locality_right': 'Nebine',
 'locality_left': 'Nebine',
 'geometry_paths': [[[147.08109996000007, -27.98588996999996],
   [147.10712904000002, -27.98704295999994]]]}

In [13]:
df = pd.DataFrame(rows)
df.sort_values(by=['road_name_full'])

,road_id,segment_id,road_name_full,alias_1_name_full,lga_name_left,lga_name_right,locality_right,locality_left,geometry_paths
0,QLDRBAL1464501528001575800,16015103990000018315,Balonne Highway,None,Paroo Shire,Paroo Shire,Nebine,Nebine,"[[[147.08109996000007, -27.98588996999996], [1..."
29,QLDRBAL1481344927963960300,16015103990000018378,Balonne Highway,None,Balonne Shire,Balonne Shire,St George,St George,"[[[148.1090241600001, -27.96231500999994], [14..."
30,QLDRBAL1481344927963960300,16015103990000018379,Balonne Highway,None,Balonne Shire,Balonne Shire,St George,St George,"[[[148.47044616000005, -28.02775202999993], [1..."
32,QLDRBAL1481344927963960300,16015103990000018381,Balonne Highway,None,Balonne Shire,Balonne Shire,St George,St George,"[[[148.56955416000005, -28.044894959999965], [..."
33,QLDRBAL1481344927963960300,16015103990000018382,Balonne Highway,None,Balonne Shire,Balonne Shire,St George,St George,"[[[148.56839604000004, -28.04473700999995], [1..."
...,...,...,...,...,...,...,...,...,...
57,QLDRVIC1485796828037130300,16015103990000376631,Victoria Street,Balonne Highway,Balonne Shire,Balonne Shire,St George,St George,"[[[148.57120692, -28.043735039999945], [148.57..."
58,QLDRVIC1485796828037130300,16015103990000376632,Victoria Street,Balonne Highway,Balonne Shire,Balonne Shire,St George,St George,"[[[148.5800499600001, -28.036859039999968], [1..."
59,QLDRVIC1485796828037130300,16015103990000376633,Victoria Street,Balonne Highway,Balonne Shire,Balonne Shire,St George,St George,"[[[148.57386012000006, -28.04166602999993], [1..."
60,QLDRVIC1485796828037130300,16015103990000376635,Victoria Street,Balonne Highway,Balonne Shire,Balonne Shire,St George,St George,"[[[148.57474104000005, -28.040984999999978], [..."


In [14]:
df.groupby(['road_name_full', 'road_id'])['road_name_full'].count()

road_name_full   road_id                   
Balonne Highway  QLDRBAL1464501528001575800    24
                 QLDRBAL1473055028002320300     6
                 QLDRBAL1481344927963960300    24
Main Street      QLDRMAI1474784328031670300     3
Victoria Street  QLDRVIC1485796828037130300     7
Name: road_name_full, dtype: int64

In [15]:
from ladb_data.models.base import LAIri
from ladb_data.models.road import RoadObject, SirIdentifier, RoadLabel, Part
from rdflib import BNode, Dataset, URIRef, RDF, SDO, Literal

from ladb_data.namespaces import CN, ROAD, RCT

# Create a top-level RoadObject with the road_id 'QLDRBAL1473055028002320300'.
# The road segments with this road_id have homogenous columns.

ROADS_GRAPH = "urn:ladb:graph:roads"
TOP_LEVEL_ROAD = URIRef("https://linked.data.gov.au/dataset/qld-addr/road-QLDRBAL1473055028002320300")
SIR_ID_DT = URIRef("urn:ladb:sir-id")
dataset = Dataset(default_graph_base=ROADS_GRAPH)
dataset.bind('cn', CN)
dataset.bind('roads', ROAD)
graph = dataset.graph(ROADS_GRAPH)

road_object = RoadObject(
    id="https://linked.data.gov.au/dataset/qld-addr/road-QLDRBAL1473055028002320300",
    sir_identifier=SirIdentifier(value="QLDRBAL1473055028002320300"),
    label=RoadLabel(
        id="https://linked.data.gov.au/dataset/qld-addr/road-label-QLDRBAL1473055028002320300",
        label="Balonne Highway",
        has_part=[
            Part(
                id=BNode().n3(),
                type=RCT.RoadName,
                value="Balonne"
            ),
            Part(
                id=BNode().n3(),
                type=RCT.RoadType,
                value=LAIri(id="https://linked.data.gov.au/def/road-types/highway")
            )
        ]
    )
)
graph += road_object.model_dump_graph()

main_st_label = RoadLabel(
    id="https://linked.data.gov.au/dataset/qld-addr/road-label-main-st",
    label="Main Street",
    has_part=[
        Part(
                id=BNode().n3(),
                type=RCT.RoadName,
                value="Main"
            ),
            Part(
                id=BNode().n3(),
                type=RCT.RoadType,
                value=LAIri(id="https://linked.data.gov.au/def/road-types/street")
            )
    ]
)
graph += main_st_label.model_dump_graph()

main_st_object = RoadObject(
    id="https://linked.data.gov.au/dataset/qld-addr/road-main-st",
    sir_identifier=SirIdentifier(value="main-st-segment"),
    type=[str(ROAD.RoadSegment), ROAD.RoadObject],
    label=main_st_label.id,
    is_part_of=road_object.id
)
graph += main_st_object.model_dump_graph()

victoria_st_label = RoadLabel(
    id="https://linked.data.gov.au/dataset/qld-addr/road-label-victoria-st",
    label="Victoria Street",
    has_part=[
        Part(
                id=BNode().n3(),
                type=RCT.RoadName,
                value="Victoria"
            ),
            Part(
                id=BNode().n3(),
                type=RCT.RoadType,
                value=LAIri(id="https://linked.data.gov.au/def/road-types/street")
            )
    ]
)
graph += victoria_st_label.model_dump_graph()

victoria_st_object = RoadObject(
    id="https://linked.data.gov.au/dataset/qld-addr/road-victoria-st",
    sir_identifier=SirIdentifier(value="victoria-st-segment"),
    type=[str(ROAD.RoadSegment), ROAD.RoadObject],
    label=victoria_st_label.id,
    is_part_of=road_object.id
)
graph += victoria_st_object.model_dump_graph()

for index, row in df.iterrows():
    if row['road_name_full'] == 'Balonne Highway':
        label = None
        is_part_of = road_object.id
    elif row['road_name_full'] == 'Main Street':
        label = main_st_label
        is_part_of = main_st_object.id
    elif row['road_name_full'] == 'Victoria Street':
        label = victoria_st_label
        is_part_of = victoria_st_object.id
    else:
        raise ValueError(f"Unknown road name {row['road_name_full']}")

    segment = RoadObject(
        id=f"https://linked.data.gov.au/dataset/qld-addr/road-segment-{row['segment_id']}",
        sir_identifier=SirIdentifier(value=row['segment_id']),
        type=[str(ROAD.RoadSegment), str(ROAD.RoadObject)],
        label=None,
        is_part_of=is_part_of
    )
    graph += segment.model_dump_graph()

for s, p, o in graph.triples((None, SDO.isPartOf, None)):
    graph.add((o, SDO.hasPart, s))

for s, p, o in graph.triples((None, SDO.name, None)):
    graph.add((o, CN.isNameFor, s))

dataset.print(format="trig")

@prefix cn: <https://linked.data.gov.au/def/cn/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix roads: <https://linked.data.gov.au/def/roads/> .
@prefix schema: <https://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<urn:ladb:graph:roads> {
    <https://linked.data.gov.au/dataset/qld-addr/road-label-QLDRBAL1473055028002320300> a cn:CompoundName,
            roads:RoadLabel ;
        rdfs:label "Balonne Highway" ;
        cn:isNameFor <https://linked.data.gov.au/dataset/qld-addr/road-QLDRBAL1473055028002320300> ;
        schema:hasPart _:Nd3248c32106a4def976988e74319ac2d,
            _:Ne75c768952e14851ba4aa90a26f0c161 .

    <https://linked.data.gov.au/dataset/qld-addr/road-label-main-st> a cn:CompoundName,
            roads:RoadLabel ;
        rdfs:label "Main Street"^^xsd:string ;
        cn:isNameFor <https://linked.data.gov.au/dataset/qld-addr/road-main-st> ;
        schema:hasPart 